<a href="https://colab.research.google.com/github/GalHillel/Final-Project-PhotoGrammetry/blob/main/pgm_p3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Initial Setup: Configuration, GPU Validation, Drive Inspection, and Cleanup

In [ ]:
# Upgrade to yt-dlp for video downloading (replacing youtube-dl)
!pip install -U yt-dlp



# Install OpenCV with a version that is compatible with GPU (CUDA support)
!pip install opencv-python-headless==4.5.4.60

# Install PyTorch to confirm GPU availability
!pip install torch torchvision torchaudio

# Install other dependencies
!pip install pysrt  # For handling SRT file parsing

In [ ]:
import shutil
import pysrt

from datetime import datetime

def backup_data():
    # Create a unique backup directory using milliseconds to avoid name collisions
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S%f")
    backup_dir = os.path.join(BACKUP_DIR, timestamp)
    os.makedirs(backup_dir, exist_ok=True)

    try:
        # Paths for specific backup folders
        state_backup = os.path.join(backup_dir, "progress_state.json")
        model_backup_dir = os.path.join(backup_dir, "models")
        frame_backup_dir = os.path.join(backup_dir, "frames")

        os.makedirs(model_backup_dir, exist_ok=True)
        os.makedirs(frame_backup_dir, exist_ok=True)

        # Copy the progress state file if it exists
        if os.path.exists(STATE_FILE_PATH):
            shutil.copy(STATE_FILE_PATH, state_backup)

        # Copy model directories with overwrite handling
        for model_folder in os.listdir(MODEL_DIR):
            model_src = os.path.join(MODEL_DIR, model_folder)
            model_dest = os.path.join(model_backup_dir, model_folder)
            if os.path.exists(model_dest):
                shutil.rmtree(model_dest)  # Remove the existing directory if necessary
            shutil.copytree(model_src, model_dest)
            logger.info(f"Backed up model directory: {model_dest}")

        # Copy frame directories with overwrite handling
        for frame_folder in os.listdir(FRAME_DIR):
            frame_src = os.path.join(FRAME_DIR, frame_folder)
            frame_dest = os.path.join(frame_backup_dir, frame_folder)
            if os.path.exists(frame_dest):
                shutil.rmtree(frame_dest)  # Remove the existing directory if necessary
            shutil.copytree(frame_src, frame_dest)
            logger.info(f"Backed up frame directory: {frame_dest}")

        logger.info("Backup completed successfully to Google Drive with timestamped directory.")

    except Exception as e:
        logger.error(f"Backup failed: {e}")


In [ ]:
import torch
import cv2
import subprocess

def validate_gpu():
    # Check GPU availability with PyTorch
    if torch.cuda.is_available():
        print(f"PyTorch GPU available: {torch.cuda.get_device_name(0)}")
        print("Number of GPUs available:", torch.cuda.device_count())
    else:
        print("No GPU detected by PyTorch.")

    # Check GPU availability for OpenCV with CUDA (requires OpenCV with CUDA build)
    try:
        gpu_count = cv2.cuda.getCudaEnabledDeviceCount()
        if gpu_count > 0:
            print(f"OpenCV GPU available: {gpu_count} CUDA-enabled device(s) detected.")
        else:
            print("No GPU detected by OpenCV with CUDA support.")
    except AttributeError:
        print("OpenCV CUDA support not found. Make sure OpenCV is installed with CUDA.")

    # Verify CUDA installation for other GPU-dependent libraries
    cuda_check = subprocess.run(['nvcc', '--version'], stdout=subprocess.PIPE, text=True)
    if cuda_check.returncode == 0:
        print("CUDA installation found:", cuda_check.stdout)
    else:
        print("CUDA not found. Please install CUDA if required for your environment.")


In [ ]:
import os
import json
import logging
from datetime import datetime, timedelta
import time
import subprocess
import cv2
import numpy as np
import shutil

# Define constants and paths
ROOT_DIR = "/content/drive/MyDrive/PhotoGrammetry"
VIDEO_DIR = os.path.join(ROOT_DIR, "videos")
FRAME_DIR = os.path.join(ROOT_DIR, "frames")
MODEL_DIR = os.path.join(ROOT_DIR, "models")
LOG_DIR = os.path.join(ROOT_DIR, "logs")
BACKUP_DIR = os.path.join(ROOT_DIR, "backups")
CONFIG_PATH = os.path.join(ROOT_DIR, "config.json")
STATE_FILE_PATH = os.path.join(ROOT_DIR, "progress_state.json")

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Load configuration
def load_config():
    if not os.path.exists(CONFIG_PATH):
        raise FileNotFoundError("Config file not found in Google Drive.")
    with open(CONFIG_PATH, 'r') as f:
        return json.load(f)
config = load_config()

# Check GPU availability
def validate_gpu():
    gpu_count = cv2.cuda.getCudaEnabledDeviceCount()
    if gpu_count > 0:
        logger.info(f"GPU available: {gpu_count} CUDA-enabled device(s) detected.")
    else:
        logger.warning("No GPU detected. Processing will proceed on CPU.")


# Check and inspect files in Google Drive
def inspect_drive():
    logger.info("Inspecting Google Drive structure...")
    for directory in [VIDEO_DIR, FRAME_DIR, MODEL_DIR, LOG_DIR, BACKUP_DIR]:
        if not os.path.exists(directory):
            os.makedirs(directory)
            logger.info(f"Created missing directory: {directory}")
        else:
            files = os.listdir(directory)
            logger.info(f"{directory} contains {len(files)} files.")



In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
print("GPU available:", torch.cuda.is_available())
print("GPU count:", torch.cuda.device_count())
inspect_drive()
validate_gpu()

GPU available: True
GPU count: 1


In [ ]:
# Path to Meshroom tar file and extraction directory
MESHROOM_TAR_PATH = "/content/drive/MyDrive/Meshroom-2023.3.0-linux.tar.gz"
MESHROOM_DIR = "/content/Meshroom-2023.3.0"

# Extract Meshroom
import tarfile
import os

if not os.path.exists(MESHROOM_DIR):
    with tarfile.open(MESHROOM_TAR_PATH, "r:gz") as tar:
        tar.extractall(path="/content/")
    print("Meshroom extracted successfully.")
else:
    print("Meshroom directory already exists.")

# Add Meshroom to system path for easy access to `meshroom_batch`
os.environ["PATH"] += os.pathsep + os.path.join(MESHROOM_DIR)

# Verify Meshroom setup
!meshroom_batch --help

Meshroom directory already exists.
usage: meshroom_batch [-h]
                      [-i [NODEINSTANCE:"SFM/FOLDERS/IMAGES;..." [NODEINSTANCE:"SFM/FOLDERS/IMAGES;..." ...]]]
                      [-I [NODEINSTANCE:"FOLDERS/IMAGES;..." [NODEINSTANCE:"FOLDERS/IMAGES;..." ...]]]
                      [-p FILE.mg/photogrammetryAndCameraTracking/cameraTrackingWithoutCalibration/cameraTracking/panoramaHdr/hdrFusion/distortionCalibration/nodalCameraTracking/stereoPhotometry/photogrammetry/panoramaFisheyeHdr/photogrammetryDraft]
                      [--overrides SETTINGS]
                      [--paramOverrides [NODETYPE:param=value NODEINSTANCE.param=value [NODETYPE:param=value NODEINSTANCE.param=value ...]]]
                      [-o FOLDER] [--cache FOLDER] [--save FILE] [--compute <yes/no>]
                      [--toNode [NODE [NODE ...]]] [--forceStatus] [--forceCompute] [--submit]
                      [--submitLabel SUBMITLABEL] [--submitter SUBMITTER]
                      [-v {,fatal

# 2. Cleanup of Incomplete or Temporary Files

In [ ]:
def cleanup_incomplete_runs():
    # Identify incomplete or temporary files and delete them
    logger.info("Performing cleanup of incomplete runs...")
    for model_folder in os.listdir(MODEL_DIR):
        model_path = os.path.join(MODEL_DIR, model_folder)
        if "incomplete" in model_folder:
            shutil.rmtree(model_path, ignore_errors=True)
            logger.info(f"Removed incomplete model data: {model_path}")
    for frame_folder in os.listdir(FRAME_DIR):
        frame_path = os.path.join(FRAME_DIR, frame_folder)
        if not os.listdir(frame_path):
            shutil.rmtree(frame_path, ignore_errors=True)
            logger.info(f"Removed empty frame directory: {frame_path}")
cleanup_incomplete_runs()

# 3. State Management for Recovery

In [ ]:
def load_state():
    if os.path.exists(STATE_FILE_PATH):
        with open(STATE_FILE_PATH, 'r') as f:
            return json.load(f)
    return {}

def save_state(state):
    with open(STATE_FILE_PATH, 'w') as f:
        json.dump(state, f, indent=4)

# Initialize state
state = load_state()

# 4. Logging and Progress Tracking

In [ ]:
def log_with_progress(message, current, total):
    percent_complete = (current / total) * 100
    logger.info(f"{message} - {percent_complete:.2f}% complete ({current}/{total})")


# 5. File Naming Helpers


In [ ]:
def get_file_name(prefix, mission_id, ext, sequence=None):
    mission_str = f"{mission_id:04d}"
    if sequence is not None:
        return f"{prefix}_{mission_str}_{sequence:04d}.{ext}"
    return f"{prefix}_{mission_str}.{ext}"


# 6. Video Download and Conversion


In [ ]:
!pip install -U yt-dlp

In [ ]:

import yt_dlp

def download_video(youtube_id, mission_id):
    try:
        video_path = os.path.join(VIDEO_DIR, get_file_name("video", mission_id, "mp4"))
        if os.path.exists(video_path):
            logger.info(f"Video already exists for mission {mission_id}. Skipping download.")
            return video_path
        ydl_opts = {
            'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]',
            'outtmpl': video_path
        }
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([f"https://www.youtube.com/watch?v={youtube_id}"])
        logger.info(f"Downloaded video for mission {mission_id}")
        return video_path
    except Exception as e:
        logger.error(f"Error downloading video {youtube_id} for mission {mission_id}: {e}")
        return None


# 7. SRT Parsing for GPS Data

In [ ]:
import re
import pysrt
def parse_dji_srt(srt_path, mission_id):
    gps_data = []
    log_entries = pysrt.open(srt_path)
    gps_pattern = re.compile(r'GPS \(([-\d.]+), ([-\d.]+)\) Altitude ([-\d.]+) m')

    for entry in log_entries:
        print(f"Parsing line: {entry.text}")  # Debugging line to inspect each entry
        match = gps_pattern.search(entry.text)
        if match:
            latitude = float(match.group(1))
            longitude = float(match.group(2))
            altitude = float(match.group(3))
            gps_data.append({
                "timestamp": entry.start.to_time().isoformat(),
                "latitude": latitude,
                "longitude": longitude,
                "altitude": altitude
            })
    if not gps_data:
        print(f"No GPS data found in {srt_path}. Check format or regex pattern.")
    return gps_data


# 8. Frame Extraction with Dynamic Sampling

In [ ]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6371e3  # Earth radius in meters
    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)
    a = np.sin(delta_phi / 2) ** 2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c

def capture_frames(video_path, mission_id, gps_data=None, base_sampling_rate=10, max_frames=1000):
    output_folder = os.path.join(FRAME_DIR, f"{mission_id:04d}")
    os.makedirs(output_folder, exist_ok=True)
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_rate = cap.get(cv2.CAP_PROP_FPS)
    saved_frames = 0
    previous_coords = None
    dynamic_sampling_rate = base_sampling_rate
    start_time = time.time()

    for i, gps_point in enumerate(gps_data):
        if saved_frames >= max_frames:
            break
        elapsed_time = time.time() - start_time
        eta = timedelta(seconds=int(elapsed_time / (saved_frames + 1) * (max_frames - saved_frames)))
        if previous_coords:
            lat1, lon1 = previous_coords
            lat2, lon2 = gps_point["latitude"], gps_point["longitude"]
            distance_moved = haversine(lat1, lon1, lat2, lon2)
            dynamic_sampling_rate = max(1, base_sampling_rate // 2 if distance_moved > 5 else base_sampling_rate * 2)
        previous_coords = (gps_point["latitude"], gps_point["longitude"])
        frame_number = int(i * dynamic_sampling_rate * frame_rate)
        if frame_number >= total_frames:
            break
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
        ret, frame = cap.read()
        if not ret:
            continue
        frame_name = get_file_name("frame", mission_id, "jpg", saved_frames)
        cv2.imwrite(os.path.join(output_folder, frame_name), frame)
        saved_frames += 1
        log_with_progress(f"Frame extraction for mission {mission_id}", saved_frames, max_frames)
        logger.info(f"ETA for mission {mission_id} - {eta}")

    cap.release()
    logger.info(f"Total frames saved for mission {mission_id}: {saved_frames}")

# 9. Meshroom Processing with GPU


In [ ]:
def run_meshroom(mission_id):
    frames_folder = os.path.join(FRAME_DIR, f"{mission_id:04d}")
    model_folder = os.path.join(MODEL_DIR, f"model_{mission_id:04d}")
    os.makedirs(model_folder, exist_ok=True)

    meshroom_log = os.path.join(model_folder, "meshroom_log.txt")

    try:
        start_time = time.time()
        result = subprocess.run(
            ["meshroom_batch", "--input", frames_folder, "--output", model_folder],
            stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True
        )
        elapsed_time = time.time() - start_time
        with open(meshroom_log, "w") as log_file:
            log_file.write(result.stdout)

        logger.info(f"Meshroom processing completed for mission {mission_id} in {timedelta(seconds=int(elapsed_time))}")
    except subprocess.CalledProcessError as e:
        logger.error(f"Meshroom encountered an error for mission {mission_id}: {e}")


# 10. Backup and Persistence

In [ ]:
def backup_data():
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S%f")  # Milliseconds for uniqueness
    backup_dir = os.path.join(BACKUP_DIR, timestamp)
    os.makedirs(backup_dir, exist_ok=True)

    try:
        model_backup_dir = os.path.join(backup_dir, "models")
        os.makedirs(model_backup_dir, exist_ok=True)

        for model_folder in os.listdir(MODEL_DIR):
            model_src = os.path.join(MODEL_DIR, model_folder)
            model_dest = os.path.join(model_backup_dir, model_folder)
            if os.path.exists(model_dest):
                shutil.rmtree(model_dest)  # Remove before overwriting
            shutil.copytree(model_src, model_dest)
            logger.info(f"Backed up model directory: {model_dest}")

        logger.info("Backup completed successfully.")

    except Exception as e:
        logger.error(f"Backup failed: {e}")


# 11. Main Orchestrator Function

In [ ]:
def setup_directories():
    for dir_path in [VIDEO_DIR, FRAME_DIR, MODEL_DIR, LOG_DIR, BACKUP_DIR]:
        os.makedirs(dir_path, exist_ok=True)
    logger.info("Directories set up successfully.")


In [ ]:
def orchestrate_process(config):
    setup_directories()

    for mission in config["missions"]:
        mission_id = mission["id"]
        state.setdefault(str(mission_id), {"downloaded": False, "frames_extracted": False, "meshroom_processed": False})

        video_path = None  # Initialize video_path to None at the start of each loop

        # Step 1: Download video
        if not state[str(mission_id)]["downloaded"]:
            video_path = download_video(mission["youtube_id"], mission_id)
            if video_path:
                state[str(mission_id)]["downloaded"] = True
                save_state(state)

        # Step 2: Parse GPS data if log file exists
        gps_data = None
        if mission.get("flight_log") and not state[str(mission_id)].get("gps_parsed"):
            log_file_path = os.path.join(LOG_DIR, mission["flight_log"])
            gps_data = parse_dji_srt(log_file_path, mission_id)
            if gps_data:  # Ensure GPS data is available
                state[str(mission_id)]["gps_parsed"] = True
                save_state(state)
            else:
                logger.warning(f"No GPS data parsed for mission {mission_id}. Skipping frame capture.")

        # Step 3: Extract frames only if video and GPS data are available
        if video_path and gps_data and not state[str(mission_id)]["frames_extracted"]:
            capture_frames(video_path, mission_id, gps_data)
            state[str(mission_id)]["frames_extracted"] = True
            save_state(state)

        # Step 4: Run Meshroom processing
        if not state[str(mission_id)]["meshroom_processed"]:
            run_meshroom(mission_id)
            state[str(mission_id)]["meshroom_processed"] = True
            save_state(state)

        # Backup and save progress at each stage
        backup_data()


In [ ]:
import json
with open(CONFIG_PATH, 'r') as f:
    config = json.load(f)

orchestrate_process(config)

Streaming output truncated to the last 5000 lines.
2024-06-19 11:13:53.999
[iso : 150] [shutter : 1/6000.0] [fnum : 170] [ev : 0] [ct : 4968] [color_md : default] [focal_len : 240] [dzoom_ratio: 10000, delta:0],[latitude: 32.105570] [longitude: 35.206576] [rel_alt: 117.600 abs_alt: 802.641] </font>
Parsing line: <font size="28">SrtCnt : 22266, DiffTime : 33ms
2024-06-19 11:13:54.032
[iso : 150] [shutter : 1/6000.0] [fnum : 170] [ev : 0] [ct : 4967] [color_md : default] [focal_len : 240] [dzoom_ratio: 10000, delta:0],[latitude: 32.105578] [longitude: 35.206580] [rel_alt: 117.600 abs_alt: 802.641] </font>
Parsing line: <font size="28">SrtCnt : 22267, DiffTime : 34ms
2024-06-19 11:13:54.065
[iso : 150] [shutter : 1/6000.0] [fnum : 170] [ev : 0] [ct : 4967] [color_md : default] [focal_len : 240] [dzoom_ratio: 10000, delta:0],[latitude: 32.105578] [longitude: 35.206580] [rel_alt: 117.600 abs_alt: 802.641] </font>
Parsing line: <font size="28">SrtCnt : 22268, DiffTime : 33ms
2024-06-19 11:13

Streaming output truncated to the last 5000 lines.
2024-06-19 11:18:14.966
[iso : 110] [shutter : 1/3000.0] [fnum : 170] [ev : 0] [ct : 5012] [color_md : default] [focal_len : 240] [dzoom_ratio: 10000, delta:0],[latitude: 32.102516] [longitude: 35.209726] [rel_alt: 117.500 abs_alt: 802.541] </font>
Parsing line: <font size="28">SrtCnt : 5173, DiffTime : 34ms
2024-06-19 11:18:14.999
[iso : 110] [shutter : 1/3000.0] [fnum : 170] [ev : 0] [ct : 5011] [color_md : default] [focal_len : 240] [dzoom_ratio: 10000, delta:0],[latitude: 32.102516] [longitude: 35.209726] [rel_alt: 117.500 abs_alt: 802.541] </font>
Parsing line: <font size="28">SrtCnt : 5174, DiffTime : 33ms
2024-06-19 11:18:15.033
[iso : 110] [shutter : 1/3000.0] [fnum : 170] [ev : 0] [ct : 5011] [color_md : default] [focal_len : 240] [dzoom_ratio: 10000, delta:0],[latitude: 32.102516] [longitude: 35.209726] [rel_alt: 117.600 abs_alt: 802.641] </font>
Parsing line: <font size="28">SrtCnt : 5175, DiffTime : 33ms
2024-06-19 11:18:15

Streaming output truncated to the last 5000 lines.
2024-06-19 11:33:52.975
[iso : 150] [shutter : 1/4000.0] [fnum : 170] [ev : 0] [ct : 4914] [color_md : default] [focal_len : 240] [dzoom_ratio: 10000, delta:0],[latitude: 32.103654] [longitude: 35.204112] [rel_alt: 119.400 abs_alt: 804.653] </font>
Parsing line: <font size="28">SrtCnt : 22634, DiffTime : 33ms
2024-06-19 11:33:53.009
[iso : 150] [shutter : 1/4000.0] [fnum : 170] [ev : 0] [ct : 4914] [color_md : default] [focal_len : 240] [dzoom_ratio: 10000, delta:0],[latitude: 32.103654] [longitude: 35.204112] [rel_alt: 119.400 abs_alt: 804.653] </font>
Parsing line: <font size="28">SrtCnt : 22635, DiffTime : 33ms
2024-06-19 11:33:53.041
[iso : 150] [shutter : 1/4000.0] [fnum : 170] [ev : 0] [ct : 4914] [color_md : default] [focal_len : 240] [dzoom_ratio: 10000, delta:0],[latitude: 32.103654] [longitude: 35.204112] [rel_alt: 119.400 abs_alt: 804.653] </font>
Parsing line: <font size="28">SrtCnt : 22636, DiffTime : 34ms
2024-06-19 11:33

Streaming output truncated to the last 5000 lines.
2024-06-19 11:35:53.249
[iso : 140] [shutter : 1/6000.0] [fnum : 170] [ev : 0] [ct : 5065] [color_md : default] [focal_len : 240] [dzoom_ratio: 10000, delta:0],[latitude: 32.103188] [longitude: 35.208155] [rel_alt: 118.400 abs_alt: 803.653] </font>
Parsing line: <font size="28">SrtCnt : 1328, DiffTime : 33ms
2024-06-19 11:35:53.283
[iso : 140] [shutter : 1/6000.0] [fnum : 170] [ev : 0] [ct : 5065] [color_md : default] [focal_len : 240] [dzoom_ratio: 10000, delta:0],[latitude: 32.103184] [longitude: 35.208166] [rel_alt: 118.400 abs_alt: 803.653] </font>
Parsing line: <font size="28">SrtCnt : 1329, DiffTime : 33ms
2024-06-19 11:35:53.316
[iso : 140] [shutter : 1/6000.0] [fnum : 170] [ev : 0] [ct : 5065] [color_md : default] [focal_len : 240] [dzoom_ratio: 10000, delta:0],[latitude: 32.103184] [longitude: 35.208166] [rel_alt: 118.400 abs_alt: 803.653] </font>
Parsing line: <font size="28">SrtCnt : 1330, DiffTime : 34ms
2024-06-19 11:35:53

Parsing line: <font size="28">SrtCnt : 2994, DiffTime : 33ms
2024-06-19 11:36:48.866
[iso : 130] [shutter : 1/6000.0] [fnum : 170] [ev : 0] [ct : 5283] [color_md : default] [focal_len : 240] [dzoom_ratio: 10000, delta:0],[latitude: 32.102606] [longitude: 35.209690] [rel_alt: 16.400 abs_alt: 701.653] </font>
Parsing line: <font size="28">SrtCnt : 2995, DiffTime : 34ms
2024-06-19 11:36:48.899
[iso : 130] [shutter : 1/6000.0] [fnum : 170] [ev : 0] [ct : 5283] [color_md : default] [focal_len : 240] [dzoom_ratio: 10000, delta:0],[latitude: 32.102606] [longitude: 35.209690] [rel_alt: 16.100 abs_alt: 701.353] </font>
Parsing line: <font size="28">SrtCnt : 2996, DiffTime : 33ms
2024-06-19 11:36:48.932
[iso : 130] [shutter : 1/6000.0] [fnum : 170] [ev : 0] [ct : 5281] [color_md : default] [focal_len : 240] [dzoom_ratio: 10000, delta:0],[latitude: 32.102606] [longitude: 35.209690] [rel_alt: 16.100 abs_alt: 701.353] </font>
Parsing line: <font size="28">SrtCnt : 2997, DiffTime : 33ms
2024-06-19 1